In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key

In [2]:
city_data = pd.read_csv("../WeatherPy1/data_vacationPy/city_list.csv") 
city_data

,Unnamed: 0,Latitude,Longitude,City,Country,Max Temp.,Humidity,Cloudiness,Windiness,Date
0,0,-1.1950,-46.0200,Carutapera,BR,82.15,78,100,7.23,1610541827
1,1,55.7435,-97.8558,Thompson,CA,26.60,100,90,3.44,1610541828
2,2,2.0167,112.9333,Kapit,MY,76.10,93,100,1.19,1610541829
3,3,-30.1811,-50.2056,Cidreira,BR,72.55,72,99,11.30,1610541653
4,4,-6.3667,37.3667,Magole,TZ,86.00,66,40,14.97,1610541831
...,...,...,...,...,...,...,...,...,...,...
530,530,-2.8856,-40.1200,Acaraú,BR,85.15,61,100,11.61,1610542517
531,531,41.5000,33.6667,Kastamonu Province,TR,37.40,93,75,8.05,1610542518
532,532,-31.9500,141.4333,Broken Hill,AU,86.00,20,100,9.22,1610542519
533,533,54.3161,-130.3201,Prince Rupert,CA,41.00,93,75,3.44,1610542376


In [3]:
gmaps.configure(api_key=g_key)

locations = city_data[["Latitude", "Longitude"]]

humid_weight = city_data['Humidity']

In [4]:
# humidity Heatmap layer
fig = gmaps.figure(zoom_level=2, center=(0,30))

#paramaters for heatmap
humid_layer = gmaps.heatmap_layer(locations, humid_weight, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)
#adding the layer to the map
fig.add_layer(humid_layer)

#display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [5]:
ideal_df =  city_data[(city_data['Max Temp.'] <= 80) & (city_data['Cloudiness'] == 0) & (city_data['Windiness'] < 10) & (city_data['Max Temp.'] >= 70)].dropna(how='any') 
ideal_df

,Unnamed: 0,Latitude,Longitude,City,Country,Max Temp.,Humidity,Cloudiness,Windiness,Date
80,80,-25.3333,-57.5333,San Lorenzo,PY,78.80,73,0,3.44,1610541931
82,82,-33.1416,-59.3097,Gualeguay,AR,77.05,44,0,8.86,1610541933
136,136,13.5928,-89.8275,Acajutla,SV,77.77,62,0,7.56,1610542002
151,151,11.2833,-15.8333,Bubaque,GW,78.58,52,0,8.81,1610542020
196,196,-33.6971,-53.4616,Chui,UY,71.28,48,0,6.06,1610542079
271,271,16.3333,-15.0000,Saint-Louis,SN,77.00,33,0,2.30,1610542174
274,274,14.7208,-92.4215,Puerto Madero,MX,76.06,63,0,2.77,1610542179
281,281,33.3406,44.4009,Baghdad,IQ,71.60,24,0,8.05,1610542062
306,306,-25.2986,152.8535,Hervey Bay,AU,72.00,83,0,3.00,1610542223
396,396,-34.4833,-54.3333,Rocha,UY,73.40,37,0,4.43,1610542339


In [6]:
hotel_df = ideal_df.loc[:,['Latitude','Longitude','City','Country']]

hotel_df['Hotel Name'] = ""

hotel_df

,Latitude,Longitude,City,Country,Hotel Name
80,-25.3333,-57.5333,San Lorenzo,PY,
82,-33.1416,-59.3097,Gualeguay,AR,
136,13.5928,-89.8275,Acajutla,SV,
151,11.2833,-15.8333,Bubaque,GW,
196,-33.6971,-53.4616,Chui,UY,
271,16.3333,-15.0000,Saint-Louis,SN,
274,14.7208,-92.4215,Puerto Madero,MX,
281,33.3406,44.4009,Baghdad,IQ,
306,-25.2986,152.8535,Hervey Bay,AU,
396,-34.4833,-54.3333,Rocha,UY,


In [7]:
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

# Iterate through 
for index, row in hotel_df.iterrows():
    
    # get lat, lng from df
    lat = row['Latitude']
    lon = row['Longitude']
    cities = row['City']

    params["location"] = f"{lat},{lon}"

# Use the search term: "Hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# make request and print url
    name_address = requests.get(base_url, params=params)
    
# convert to json
    response = requests.get(base_url, params=params).json()
    results = response['results']

   # Grab the first hotel from the results and store the name
    try:
        print(f"First hotel nearest to {cities} is {results[0]['name']}.")
        
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
    except(KeyError, IndexError):
        print("Nothing here! I guess you won't be staying here")
        
    time.sleep(1)



First hotel nearest to San Lorenzo is Canarias Hotel.


NameError: name 'time' is not defined

In [ ]:
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display figure
fig